In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Fri Feb 28 13:42:57 PST 2020
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.931
BogoMIPS:              5851.96
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [7]:
# Set workding directory
%env wd=/home/sam/analyses/20200228_swoose_olur_v081_fasta_renaming
wd="/home/sam/analyses/20200228_swoose_olur_v081_fasta_renaming"

%env rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate  https://owl.fish.washington.edu/halfshell/genomic-databank/

%env og_fasta=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.fasta
%env og_fai=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.fasta.fai
%env og_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
%env gene_gff=Olurida_v081-20190709.gene.gff
%env new_fasta=Olurida_v081.genes.fasta
%env new_fai=Olurida_v081.genes.fasta.fai
%env samtools=/home/sam/programs/samtools-1.9/samtools

env: wd=/home/sam/analyses/20200228_swoose_olur_v081_fasta_renaming
env: rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
env: wget_command=--directory-prefix=$/home/sam/analyses/20200228_swoose_olur_v081_fasta_renaming --quiety --no-directories --no-check-certificate  https://owl.fish.washington.edu/halfshell/genomic-databank/
env: og_fasta=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.fasta
env: og_fai=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.fasta.fai
env: og_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
env: gene_gff=Olurida_v081-20190709.gene.gff
env: new_fasta=Olurida_v081.genes.fasta
env: new_fai=Olurida_v081.genes.fasta.fai
env: samtools=/home/sam/programs/samtools-1.9/samtools


#### Create necessary directories

In [8]:
%%bash
mkdir --parents ${wd}

In [9]:
cd {wd}

/home/sam/analyses/20200228_swoose_olur_v081_fasta_renaming


#### Download Olur_v081 genes gff

Taken from: https://owl.fish.washington.edu/halfshell/genomic-databank/

In [11]:
%%bash
# Create array of files from list
files_array=(Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff Olurida_v081-20190709.gene.gff)


for file in "${files_array[@]}"
do
  rsync \
  --archive \
  --progress \
  --verbose \
  "${rsync_owl}${file}" \
  .
done

ls -lh

receiving incremental file list
Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
  3,104,658,743 100%   29.57MB/s    0:01:40 (xfr#1, to-chk=0/1)

sent 30 bytes  received 3,105,037,889 bytes  30,591,506.59 bytes/sec
total size is 3,104,658,743  speedup is 1.00
receiving incremental file list

sent 11 bytes  received 75 bytes  57.33 bytes/sec
total size is 9,248,086  speedup is 107,535.88
total 3.0G
-rw-r--r-- 1 sam users 8.9M Jul 16  2019 Olurida_v081-20190709.gene.gff
-rw-rw-r-- 1 sam users 2.9G Dec 13 21:30 Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff


### Extract FastA from MAKER GFF

In [12]:
%%bash

# Find the first line in the file that begins with ">"
# and print to the end of the file ('p' enables printing the first matching line, which would be skipped by default)
sed --quiet '/^>/,$p' ${og_gff} > ${og_fasta}

#### Compare number of FastA header lines

In [13]:
%%bash

grep --with-filename --count "^>" ${og_gff}
grep --with-filename --count "^>" ${og_fasta}

Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff:159429
Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.fasta:159429


#### Create FastA index file

In [14]:
%%bash

${samtools} faidx ${og_fasta} > ${og_fai}

head ${og_fai}

Contig56127	12532	13	60	61
Contig81695	22750	12767	60	61
Contig130560	1525	35911	60	61
Contig155059	2121	37476	60	61
Contig61093	10407	39646	60	61
Contig79811	2248	50240	60	61
Contig89862	14043	52539	60	61
Contig1111	28792	66829	60	61
Contig160984	4700	96115	60	61
Contig214118	1068	100908	60	61


### Change scaffold names and file names

In [17]:
%%bash

# Array of old scaffold names
# Uses first column of GFF to get scaffold names
mapfile -t orig_scaffold_names < <(sort -k1,1 -t $'\t' ${gene_gff} | awk '{print $1}')

# Array of new scaffold names
# Separators set as "=" and ";" to pull out new IDs
# NR > 1 skips the first line (i.e. header)
mapfile -t new_scaffold_names < <(sort -k1,1 -t $'\t' ${gene_gff} | awk -F"[=;]" 'NR > 1 {print $2}')

  # sed substitution
  # creates sed script to find original scaffold names and replace them with new scafold names
  # and passes to sed via stdin
# the "\<\>" in the printf is needed for exact sed match
for index in "${!orig_scaffold_names[@]}"
  do
    printf "s/%s/%s/\n" "\<${orig_scaffold_names[index]}\>" "${new_scaffold_names[index]}"
  done | sed --file - "${og_fasta}" \
  >> "${new_fasta}"
done

ls -ltrh

Process is interrupted.


#### Check old and new FastAs to confirm substituions

In [19]:
%%bash

grep --with-filename "^>" ${og_fasta} | head

grep --with-filename "^>" ${new_fasta} | head

##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	2	125	.	+	0	ID=PGEN_.00g000010.m01.CDS01;Name=PGEN_.00g000010.m01.CDS01;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	1995	2095	.	+	1	ID=PGEN_.00g000010.m01.CDS02;Name=PGEN_.00g000010.m01.CDS02;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	3325	3495	.	+	0	ID=PGEN_.00g000010.m01.CDS03;Name=PGEN_.00g000010.m01.CDS03;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.00g234140.m01
Scaffold_01	GenSAS_5d9637f372b5d-publish	CDS	4651	4719	.	+	0	ID=PGEN_.00g000010.m01.CDS04;Name=PGEN_.00g000010.m01.CDS04;Parent=PGEN_.00g000010.m01;original_ID=cds.21510-PGEN_.00g234140.m01;Alias=cds.21510-PGEN_.0

#### Make FastA index file for new FastA

In [20]:
%%bash

${samtools} faidx ${new_fasta} > ${new_fai}

head ${new_fai}

total 3.7G
-rw-rw-r-- 1 sam users  53M Nov  5 08:55 Panopea-generosa-v1.0.a4.CDS.gff3
-rw-rw-r-- 1 sam users  55M Nov  5 08:55 Panopea-generosa-v1.0.a4.exon.gff3
-rw-rw-r-- 1 sam users 9.5M Nov  5 08:55 Panopea-generosa-v1.0.a4.gene.gff3
-rw-rw-r-- 1 sam users 9.1M Nov  5 08:55 Panopea-generosa-v1.0.a4.mRNA.gff3
-rw-rw-r-- 1 sam users 326M Nov  5 08:55 Panopea-generosa-v1.0.a4.repeat_region.gff3
-rw-rw-r-- 1 sam users 1.2K Nov  5 08:55 Panopea-generosa-v1.0.a4.rRNA.gff3
-rw-rw-r-- 1 sam users 2.3M Nov  5 08:55 Panopea-generosa-v1.0.a4.tRNA.gff3
-rw-rw-r-- 1 sam users 914M Nov  5 08:56 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam users  658 Nov  5 08:56 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam users 2.0G Nov  5 08:59 Panopea-generosa-v1.0.CpG.gff
-rw-rw-r-- 1 sam users 996K Nov  5 08:59 Panopea-generosa-v1.0.a4.intergenic.bed
-rw-rw-r-- 1 sam users 4.4M Nov  5 08:59 Panopea-generosa-v1.0.a4.introns.bed
-rw-rw-r-- 1 sam users 4.7M Nov  5 08:59 Panopea-generosa-v1.0.a4.repeats.DNA.gff3
